# stacking phase

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network

prj_dir = '/media/ly/文件/sc2/mintpy'
if not os.path.isdir(prj_dir):
    os.mkdir(prj_dir)

os.chdir(prj_dir)                        # work directory
prj_file = 'Sen_stacking.txt'                # configure file

##  configure file

In [ ]:
config = """##-------------------------------- MintPy -----------------------------##
########## 1. Load Data (--load to exit after this step)
## load_data.py -H to check more details and example inputs.
mintpy.load.processor      = gamma
mintpy.load.unwFile        = ./interferograms/*/diff_*rlks.unw
mintpy.load.corFile        = ./interferograms/*/filt_*rlks.cor
mintpy.load.connCompFile   = None
mintpy.load.intFile        = None

mintpy.load.demFile        = ./geom_master/sim_*.rdc.dem
mintpy.load.lookupYFile    = ./geom_master/sim_*.UTM_TO_RDC
mintpy.load.lookupXFile    = ./geom_master/sim_*.UTM_TO_RDC
mintpy.load.incAngleFile   = None
mintpy.load.azAngleFile    = None
mintpy.load.shadowMaskFile = None
mintpy.load.waterMaskFile  = None
##---------subset (optional):
mintpy.subset.yx 			= 0:1300,0:1200
########## 2. modify_network
mintpy.network.coherenceBased  = yes  #[yes / no], auto for no, exclude interferograms with coherence < minCoherence
mintpy.network.keepMinSpanTree = no  #[yes / no], auto for yes, keep interferograms in Min Span Tree network
mintpy.network.minCoherence    = 0.6  #[0.0-1.0], auto for 0.7
"""
with open(prj_file, 'w+') as f:
    f.write(config)

## load data

In [ ]:
!smallbaselineApp.py $prj_file --dostep load_data

## modify network

In [ ]:
!smallbaselineApp.py $prj_file --dostep modify_network

## reference point

In [ ]:
!smallbaselineApp.py $prj_file --dostep reference_point

## average

In [ ]:
!temporal_average.py ./inputs/ifgramStack.h5 -d unwrapPhase -o avgPhaseVelocity.h5

## geocode

In [ ]:
!geocode.py -l ./inputs/geometryGeo.h5 -t $prj_file --outdir ./geo_avg --update ./avgPhaseVelocity.h5
!geocode.py -l ./inputs/geometryGeo.h5 -t $prj_file --outdir ./geo_avg --update ./inputs/geometryRadar.h5

In [ ]:
view.main('./geo_avg/geo_avgPhaseVelocity.h5 velocity --dem ./geo_avg/geo_geometryRadar.h5 --figsize 12 12 --notick --noaxis'.split())

## save kmz

In [ ]:
!save_kmz.py ./geo_avg/geo_avgPhaseVelocity.h5 -c jet -u mm -v -60 60 -o ./geo_avg/geo_avgPhaseVelocity --dpi 600